## Notebook to check the CpG sites indentifed in EWAS of PD/LBD Lewy Body Pathology in FOUNDIN-PD Epigentic Analyses

Pihlstrøm L, Shireby G, Geut H et al. Epigenome-wide association study of human frontal cortex identifies differential methylation in Lewy body pathology. Nat Commun 2022;13:4932.
https://pubmed.ncbi.nlm.nih.gov/35995800/

In [1]:
!date

Tue Feb 28 16:17:43 UTC 2023


#### import libraries

In [2]:
import statsmodels.stats.multitest as smm
from pandas import DataFrame, read_parquet, concat
import concurrent.futures
from os.path import exists

#### set notebook variables

In [3]:
# naming

# directories
wrk_dir = '/home/jupyter/foundin_qtl'
tensorqtl_dir = f'{wrk_dir}/tensorqtl'

# input files
qtl_file_frmt = '{dir}/foundin_{day}_{pair}.cis_qtl_pairs.chr{chrom}.parquet'

# constants
cpg_sites = ['cg07107199', 'cg14511218', 'cg09985192', 'cg04011470']
# meth data is only da0 and da65
days = ['da0', 'da65']
result_pairs = ['ATAC-METH', 'CIRC-METH', 'PDUI-METH', 'RNAB-METH', 'RNAS-METH']
alpha_value = 0.05
DEBUG = True
AUTOSOMES = [str(x) for x in list(range(1,23))]

#### utility functions

In [4]:
# compute B&H FDR for given p-values
def compute_fdr(pvalues):
    bh_adj = smm.fdrcorrection(pvalues)
    return bh_adj[1]

def read_qtl_results(in_file: str) -> DataFrame:
    qtl_df = read_parquet(in_file)
    qtl_df['cispair'] = qtl_df['phenotype_id'] + ':' + qtl_df['variant_id']
    return qtl_df

def read_all_qtl_results(day: str, pair: str, verbose: bool=False) -> DataFrame:
    fs_list = []
    lm_results = []
    with concurrent.futures.ThreadPoolExecutor() as tpe:
        for chrom in AUTOSOMES:
            this_result_file = qtl_file_frmt.format(dir=tensorqtl_dir, day=day, 
                                                    pair=pair, chrom=chrom)
            if exists(this_result_file):
                fs_list.append(tpe.submit(read_qtl_results, this_result_file))
    for future in concurrent.futures.as_completed(fs_list):
        lm_results.append(future.result()) 
    # combine the read results
    qtl_df = concat(lm_results)    
    print(f'{pair} qtl results shape {qtl_df.shape}')
    if verbose:
        display(qtl_df.sample(5))
    return qtl_df

### check significant results for CpG sites of interest

for each day in each analysis pairing

In [5]:
%%time
for pairing in result_pairs:
    for day in days:
        print(day, pairing)
        # load results
        results_df = read_all_qtl_results(day, pairing, verbose=DEBUG)
        # apply B&H FDR corrections to results
        results_df['bh_fdr'] = compute_fdr(results_df['pval_nominal'].fillna(1))
        # check significant results for CpG sites of interest
        sig_reuslts = results_df.loc[(results_df.variant_id.isin(cpg_sites)) & (results_df.bh_fdr <= alpha_value)]
        print(sig_reuslts.shape)
        display(sig_reuslts.variant_id.value_counts())
        print(sig_reuslts.cispair.unique())
        if DEBUG:
            if sig_reuslts.shape[0] < 50:
                display(sig_reuslts)
            else:
                display(sig_reuslts.sample(10))        

da0 ATAC-METH
ATAC-METH qtl results shape (121619951, 10)


,phenotype_id,variant_id,tss_distance,af,ma_samples,ma_count,pval_nominal,slope,slope_se,cispair
2768922,chr20_48143074_48144210,cg06398873,-649424,0.251038,48,28,0.108796,-0.176782,0.109203,chr20_48143074_48144210:cg06398873
1247808,chr3_156174326_156174817,cg05511503,-429124,0.237217,34,20,0.739332,0.033600,0.100685,chr3_156174326_156174817:cg05511503
3656805,chr6_3211056_3212337,cg00101728,-258263,0.234398,32,18,0.808074,-0.023447,0.096258,chr6_3211056_3212337:cg00101728
5017117,chr8_17805735_17806329,cg18736775,585404,0.254101,50,29,0.358748,0.114971,0.124667,chr8_17805735_17806329:cg18736775
5662289,chr6_35341430_35343683,cg08207604,-43899,0.238253,44,25,0.126059,-0.155297,0.100620,chr6_35341430_35343683:cg08207604


(0, 11)


Series([], Name: variant_id, dtype: int64)

[]


...

(0, 11)


Series([], Name: variant_id, dtype: int64)

[]


,phenotype_id,variant_id,tss_distance,af,ma_samples,ma_count,pval_nominal,slope,slope_se,cispair,bh_fdr


CPU times: user 8min 40s, sys: 2min 52s, total: 11min 32s
Wall time: 1h 2min


### check that a single ATAC site for correlation with RNAB

In [7]:
# %%time
# this_result_file = qtl_file_frmt.format(dir=tensorqtl_dir, day='da65', 
#                                         pair='RNAB-ATAC', chrom=1)
# temp_df = read_parquet(this_result_file)
# print(temp_df.shape)
# probe_results = temp_df.loc[temp_df.variant_id == 'chr1_205042708_205045441']
# print(probe_results.shape)
# display(probe_results.sort_values('pval_nominal'))

In [8]:
!date

Tue Feb 28 20:33:06 UTC 2023
